In [1]:
import os
os.chdir('C:\\Projects\\ature')
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import neuralnet.utils.data_utils as dutils

Using TensorFlow backend.


### Directories setup

In [12]:

sep = os.sep
Dirs = {}
Dirs['train_data']      = 'data'+sep+'DRIVE'+sep+'training'+sep +'patches'

Dirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
Dirs['images']    = Dirs['data'] +sep+ 'images'
Dirs['mask']      = Dirs['data'] +sep+ 'mask'
Dirs['truth']     = Dirs['data'] +sep+ '1st_manual'
Dirs['segmented'] = Dirs['data'] +sep+ 'drive_segmented'
Dirs['test_data'] = Dirs['data'] +sep+ 'patches'
Dirs['checkpoint']   = 'data' +sep+ 'checkpoint'

for k, folder in Dirs.items():
    os.makedirs(folder, exist_ok=True)
    
batch_size = 32
num_classes = 4
epochs = 5

### Load training and test data

In [14]:
x_train, y_train = dutils.load_dataset(data_path=Dirs['train_data'], img_shape=(31,31,1), num_classes=num_classes)
x_test, y_test = dutils.load_dataset(data_path=Dirs['test_data'], img_shape=(31,31,1), num_classes=num_classes)

class_weights = dutils.get_class_weights(y_train)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)
datagen.fit(x_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

Data file loaded: data\DRIVE\training\patches\21_training.npy
Data file loaded: data\DRIVE\training\patches\22_training.npy
Data file loaded: data\DRIVE\training\patches\23_training.npy
Data file loaded: data\DRIVE\training\patches\24_training.npy
Data file loaded: data\DRIVE\training\patches\25_training.npy
Data file loaded: data\DRIVE\test\patches\26_training.npy
Data file loaded: data\DRIVE\test\patches\27_training.npy


### Define model

In [6]:
model = Sequential() 

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(31,31,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              class_weight=class_weights)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)